In [1]:
from brt.cpp.router import generate_local_route_indices
import torch

dst_num = 4
gates = torch.randn((4, 4)).cuda()
topk_indices = torch.topk(gates, k=2, dim=1).indices

route_indices = (
    torch.zeros(
        gates.size(0),
        dst_num,
        dtype=torch.int32,
        device=gates.device,
    )
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()

local_indices, dst_loads = generate_local_route_indices(
    route_indices, supported_capacities
)
print(local_indices)
local_indices = local_indices * route_indices
print(local_indices)
print(dst_loads)




tensor([[1, 1, 0, 0],
        [2, 1, 1, 0],
        [2, 2, 1, 1],
        [3, 2, 2, 1]], device='cuda:0', dtype=torch.int32)
tensor([[1, 1, 0, 0],
        [2, 0, 1, 0],
        [0, 2, 0, 1],
        [3, 0, 2, 0]], device='cuda:0', dtype=torch.int32)
tensor([4, 2, 2, 1], device='cuda:0', dtype=torch.int32)


In [3]:
from brt.cpp.router import route_data_with_local_indices
import torch

in_data = torch.randn(4, 4, dtype=torch.float32).cuda()

total_load = torch.sum(dst_loads).item()
out_data = torch.zeros((total_load, 4), dtype=in_data.dtype).cuda()

new_gates = torch.ones_like(gates, dtype=torch.float32).cuda()

new_gates = new_gates / 2

out_data = route_data_with_local_indices(
    in_data,
    out_data,
    new_gates,
    local_indices,
    dst_loads,
)
print(in_data)
print(out_data)


tensor([[-0.3510, -0.7133,  0.5474,  1.1233],
        [ 0.5072, -0.6250,  1.7217, -0.6054],
        [ 1.0321,  0.6146, -1.2543, -0.5825],
        [ 1.8319,  0.0345,  0.4426,  0.5731]], device='cuda:0')
tensor([[-0.1755, -0.3567,  0.2737,  0.5616],
        [ 0.2536, -0.3125,  0.8609, -0.3027],
        [ 0.9159,  0.0173,  0.2213,  0.2866],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1755, -0.3567,  0.2737,  0.5616],
        [ 0.5160,  0.3073, -0.6272, -0.2913],
        [ 0.2536, -0.3125,  0.8609, -0.3027],
        [ 0.9159,  0.0173,  0.2213,  0.2866],
        [ 0.5160,  0.3073, -0.6272, -0.2913]], device='cuda:0')
